# Binary Classification of 747 vs A380
The goal is to classify a picture between two classes : an Boeing 747 or an Airbus A380.
*Spoiler :* This source code is 3 times better than Google AutoML on the same dataset !

## Dataset
It's an homemade dataset from Google Image.
There is a nice script to get images from Google : https://github.com/boxabirds/fastai-helpers/blob/master/training-data-generator.py, it's based on https://github.com/hardikvasa/google-images-download wich use ChromeDriver to automate the process.

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
PATH = "../../datasets/boeing_vs_airbus/"
!rm -r {PATH}tmp
## Getting dataset
#!mkdir {PATH}
#!pip install google_images_download
#!ls ../../datasets/
#!cd {PATH} && mkdir -p train/747 train/A380 test/747 test/A380  valid/747 valid/A380
#!cd {PATH}/747 && googleimagesdownload --keywords "Boing 747" --limit 1000 --chromedriver /usr/local/bin/chromedriver
#!cd {PATH}/A380 && googleimagesdownload --keywords "Airbus A380" --limit 1000 --chromedriver /usr/local/bin/chromedriver
## Getting models weights
#!cd ./fastai && wget http://files.fast.ai/models/weights.tgz && tar -xvzf weights.tgz
#!ls {PATH}

rm: cannot remove '../../datasets/boeing_vs_airbus/tmp': No such file or directory


In [3]:
print("Setting CUDA devices...")
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="6"
print("Loading Fast.AI modules...")
# This file contains all the main external libs we'll use
from fastai import *
from fastai.vision import *
# from fastai.transforms import *
# from fastai.conv_learner import *
# from fastai.model import *
# from fastai.dataset import *
# from fastai.sgdr import *
# from fastai.plots import *
# from sklearn.metrics import confusion_matrix

sz=299
arch=models.resnet50
#arch=resnet34
bs=132


Setting CUDA devices...
Loading Fast.AI modules...


In [4]:
print("CuDNN optimization is ", torch.backends.cudnn.enabled)

CuDNN optimization is  True


# Training from sratch
We begin with an empty Resnet50 model.

## Computing mean and stddev

First we compute the mean and standard deviation of all images to center them.
It will help calculus to have a mean of 0 and a standard deviation of 1.

In [5]:
import glob
import PIL
means = np.array([0.0,0.0,0.0])
files = [val for sublist in [[os.path.join(i[0], j) for j in i[2]] for i in os.walk(f'{PATH}train')] for val in sublist]
print("Input images number : ", len(files))
deviations = np.array([0.0,0.0,0.0])
#variances = np.array([0,0,0])
variance = np.array([0.0,0.0,0.0])
for image in files:
    #print(image)
    img_stats = PIL.ImageStat.Stat(PIL.Image.open(image), mask=None)
    means += np.array(img_stats.mean)
    variance += np.array(img_stats.var)

stddev = np.sqrt(variance) / len(files)
print("stddev with PIL : ", stddev)

mean = np.array(means) / len(files) / 255
print("Means with PIL : ", mean)
stats = (mean, stddev)

Input images number :  1204
stddev with PIL :  [1.61072035 1.57198706 1.66764579]
Means with PIL :  [0.53317908 0.56569031 0.59960121]


_23 october note :_ In the new version of fastai librairy you could do it in one line :

In [6]:
data.normalize(imagenet_stats)

TypeError: normalize() missing 2 required positional arguments: 'mean' and 'std'

### Prepare data and model

In [ ]:
#transform.
data = ImageDataBunch.from_folder(PATH, ds_tfms=get_transforms(), size=224)
data.normalize(imagenet_stats)
data.show_batch(rows=3, figsize=(7,6))
learn = ConvLearner(data, arch, metrics=error_rate) #accuracy

In [ ]:
#learn.summary()
data.classes

The model is realy deep as it make use of 168 layers.

How much input data do we get ?

In [ ]:
# This is the label for a val data
# plt.hist(data.train_ds.tfm_y)
# plt.hist(data.val_y)

In [ ]:
# size_d = {k: PIL.Image.open(PATH+k).size for k in data.trn_ds.fnames}
# row_sz, col_sz = list(zip(*size_d.values()))
# row_sz = np.array(row_sz)
# col_sz = np.array(col_sz)
# plt.hist(row_sz);
# plt.hist(col_sz);

In [ ]:
# plt.hist(row_sz[row_sz < 1000])

We have few data, but sizes are good.

### Guess learning rate

In [ ]:
lrf=learn.lr_find(1e-6, end_lr=1)
#learn.sched.plot_lr()

In [ ]:
learn.recorder.plot()

In [ ]:
Unfortunatly the learning rate finder do not work well with small dataset.
_23 october note :_ In the new version of fastai librairy it seems to work !

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(5,5), dpi=60)

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
#F2 :  0.58 for lr=1e-2, image_size=128, batch_size=150, epoch=30 # val_loss begin at 4.4 and acc stay at 0.50 
#F2 :  0.02 for lr=1e-3, image_size=128, batch_size=256, epoch=30 # very erratic graph
#F2 :  0.55 for lr=1e-4, image_size=128, batch_size=256, epoch=30
#F2 :  0.11 for lr=1e-5, image_size=128, batch_size=256, epoch=30
#F2 :  0.55 for lr=1e-4, image_size=128, batch_size=128, epoch=30
#F2 :  0.49 (56.12 %) for lr=1e-4, image_size=128, batch_size=64
#F2 :  0.55 (49.35 %) for lr=1e-4, image_size=128, batch_size=64

lr=1e-4
image_size=128
batch_size=64
epoch=30
#data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, image_size), bs=batch_size, num_workers=num_cpus())
data = ImageDataBunch.from_folder(PATH, ds_tfms=get_transforms(do_flip=False, flip_vert=False, max_rotate=0, max_zoom=0), size=224) # ds_tfms=get_transforms()
data.normalize(imagenet_stats)


In [ ]:
learn = ConvLearner(data=data, arch=arch, ps=0.5, pretrained=False, metrics=accuracy)
learn.fit(epochs=epoch, lr=lr) # ,  get_ep_vals=True

In [ ]:
#print("After ", str(len(global_results)), " epochs, the accuracy is ", str(vals_s2s[1]*100)[:5], "%" )
#plot_ep_vals(global_results)
#plot_the_confusion_matrix()
learn.recorder.plot()

In [ ]:
learn.recorder.plot_lr()

In [ ]:
learn.recorder.plot_metrics()

In [ ]:
learn.recorder.plot_losses()

The model had learn quite nothing. But it is not surprising with so small dataset.

## Fine tuning

In [ ]:
!rm -r {PATH}tmp

#F2 :  0.83 for image_size=128, batch_size=128, lr=1e-2
# F2 :  0.80 for image_size=128, batch_size=150, lr=1e-3
# F2 :  0.76 for image_size=128, batch_size=150, lr=1e-4 # good graph
# F2 :  0.83 for image_size=128, batch_size=150, lr=1e-2 # val loss goes up => overfitting
# F2 :  0.81 (acc 81.61 %) for image_size=128, batch_size=150, lr=1e-3 # val_loss go down, then slowly up
# F2 :  0.76 (acc 76.45 %) for image_size=128, batch_size=64, lr=1e-4 # good graph, val_loss slowly go down
# F2 :  0.76 (acc 77.74 %) for image_size=128, batch_size=64, lr=1e-4 # good graph, val_loss slowly go down

image_size=128
batch_size=64
lr=1e-4
epoch=30

data = ImageDataBunch.from_folder(PATH, ds_tfms=get_transforms(do_flip=False, flip_vert=False, max_rotate=0, max_zoom=0), size=224) # ds_tfms=get_transforms()
data.normalize(imagenet_stats)
learn = ConvLearner(data=data, arch=arch, ps=0.5, pretrained=True, metrics=accuracy)
learn.unfreeze()
learn.fit(epochs=epoch, lr=lr)

In [ ]:
learn.recorder.plot_losses()

Fine tunning realy improve the accuracy, it is the first thing to do with any project.

## Data augmentation
Apply random operations on pictures to help the model to generalize.
Data augmentation options : https://becominghuman.ai/data-augmentation-using-fastai-aefa88ca03f1

In [ ]:
# aug_tfms=[RandomRotate(10, tfm_y=TfmType.NO),
#         RandomFlip(),
#         RandomLighting(0.05, 0.05,tfm_y=TfmType.NO),
#         RandomZoom(zoom_max=0.2),
#         RandomStretch(max_stretch=0.2)]
#tfms = tfms_from_model(arch, image_size, aug_tfms=aug_tfms, max_zoom=1.1)
tfms = get_transforms(do_flip=True, flip_vert=False, max_rotate=10, max_zoom=1.1)


# def get_augs():
#     data = ImageDataBunch.from_folder(PATH, ds_tfms=tfms, size=224) # ds_tfms=get_transforms()
#     x,_ = next(iter(data.train_dl))
#     return data.train_dl.denorm(x)[1]

# ims = np.stack([get_augs() for i in range(8)])
# plots(ims, rows=2)

In [ ]:
!rm -r {PATH}tmp
# F2 :  0.77 for image_size=128, batch_size=64, lr=1e-2 # val_loss don't go down
# F2 :  0.83 for image_size=128, batch_size=128, lr=1e-3, epoch=30
# F2 :  0.75 for image_size=128, batch_size=256, lr=1e-4, epoch=30
# F2 :  0.81 for image_size=128, batch_size=64, lr=1e-4 # good graph
# F2 :  0.85 (acc 84.19 %) for image_size=128, batch_size=64, lr=1e-3 # val_loss go slowy up
# F2 :  0.79 (acc 76.45 %) for image_size=128, batch_size=64, lr=1e-4 # val_loss still go down
# F2 :  0.84 (acc 85.16 %) for image_size=128, batch_size=64, lr=1e-3 # val_loss go slowy up

image_size=128
batch_size=64
lr=1e-3
epoch=30


data = ImageDataBunch.from_folder(PATH, ds_tfms=tfms, size=224) # ds_tfms=get_transforms()
data.normalize(imagenet_stats)
learn = ConvLearner(data=data, arch=arch, ps=0.5, pretrained=True, metrics=accuracy)
learn.unfreeze()
learn.fit(epochs=epoch, lr=lr)

# data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=batch_size, num_workers=num_cpus())
# learn = ConvLearner.pretrained(arch, data, precompute=False, ps=0.5, pretrained=True)
# learn.unfreeze()
# vals_s2s, global_results = learn.fit(lr, n_cycle=epoch, get_ep_vals=True)
# print("After ", str(len(global_results)), " epochs, the accuracy is ", str(vals_s2s[1]*100)[:5], "%" )
# plot_ep_vals(global_results)
# plot_the_confusion_matrix()

In [ ]:
learn.recorder.plot_losses()

Using data augmentation improve the accuracy by 10 points.

## Multiple size

In [ ]:
!rm -r {PATH}tmp
#F2 :  0.82 for learning_rate = 1e-2, dropout = [0.25,0.5], image_size = 128, batch_size = 150 # val_loss goes up
#F2 :  0.83 for learning_rate = 1e-2, dropout = 0.5, image_size = 128, batch_size = 150
#F2 :  0.61 for learning_rate = 1e-4, dropout = 0.5, image_size = 128, batch_size = 150 # bad graph
#F2 :  0.78 (acc 76.45 %) for learning_rate = 1e-3, dropout = [0.25,0.5], image_size = 128, batch_size = 150 # good graph
#F2 :  0.81 (acc 82.25 %) for learning_rate = 1e-3, dropout = [0.25,0.5], image_size = 128, batch_size = 64 # good, val_loss goes down
#F2 :  0.84 (acc 82.58 %) for learning_rate = 1e-2, dropout = [0.25,0.5], image_size = 128, batch_size = 64 # better acc but val_loss goes up


learning_rate = 1e-3
dropout = 0.5 #[0.25,0.5]
image_size = 128
batch_size = 64


data = ImageDataBunch.from_folder(PATH, ds_tfms=tfms, size=224) # ds_tfms=get_transforms()
data.normalize(imagenet_stats)
learn = ConvLearner(data=data, arch=arch, ps=0.5, pretrained=True, metrics=accuracy)
learn.unfreeze()

training_loop = [
    [123, 64, 10],
    [150, 128, 10],
    [123, 224, 10],
]
for bs, sz, epoch in training_loop:
    data.batch_size = bs
    learn.fit(epochs=epoch, lr=lr)

# global_results = collections.OrderedDict([])
# aug_tfms=[RandomRotate(10, tfm_y=TfmType.NO), RandomFlip(), RandomLighting(0.05, 0.05,tfm_y=TfmType.NO), RandomZoom(zoom_max=0.2),RandomStretch(max_stretch=0.2)]
# tfms = tfms_from_model(arch, image_size, aug_tfms=aug_tfms, max_zoom=1.1)
# data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=batch_size, num_workers=num_cpus())
# learn = ConvLearner.pretrained(arch, data, precompute=False, ps=dropout, pretrained=True)
# learn.unfreeze()
# training_loop = [
#     [123, 300, 1],
#     [512, 64, 10],
#     [150, 128, 10],
#     [123, 300, 10],
# ]

# for bs, sz, epoch in training_loop:
#     lr=np.array([learning_rate/100,learning_rate/10,learning_rate]) # Learning rate plus faible pour les premières couche, pour ré-apprendre un peu si necessaire
#     tfms = tfms_from_model(arch, sz, aug_tfms=aug_tfms, max_zoom=0.5)
#     data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=num_cpus())
#     vals_s2s, ep_vals_s2s = learn.fit(lr, n_cycle=epoch, get_ep_vals=True)
#     if len(global_results) > 0:
#         #print(global_results)
#         for k, v in ep_vals_s2s.items():
#             global_results[len(global_results)] = v
#     else:
#         global_results = ep_vals_s2s
#     print("After ", str(len(global_results)), " epochs, the accuracy is ", str(vals_s2s[1]*100)[:5], "%" )
#     fichier = arch.__name__ + '_' + str(len(global_results)) + "_" + str(sz) + "_acc" + str(vals_s2s[1]*100)[:5] + '_weights'
#     print("Saving to ", fichier)
#     learn.save(fichier)
# plot_ep_vals(global_results)
# plot_the_confusion_matrix()

In [ ]:
learn.recorder.plot_losses()

Well, using multiple size did not realy help but it is something to try.

## SGDR - Learning rate anealing

In [ ]:
learn.recorder.plot()

In [ ]:
!rm -r {PATH}tmp

# SGDR : Earlier we said 3 is the number of epochs, but it is actually *cycles*. So if cycle_len=2 , it will do 3 cycles where each cycle is 2 epochs (i.e. 6 epochs). Then why did it 7? It is because of cycle_mult :
#     cycle_mult=2 : this multiplies the length of the cycle after each cycle (1 epoch + 2 epochs + 4 epochs = 7 epochs).

# F2 :  0.81 for learning_rate = 1e-4, lr=np.array([learning_rate/100,learning_rate/10,learning_rate]), dropout = [0.25,0.5] 
# F2 :  0.91 for learning_rate = 1e-4, lr=np.array([learning_rate/5,learning_rate/3,learning_rate]), dropout = [0.25,0.5] # good graph
# F2 :  0.93 (acc 94.51 %) for learning_rate = 1e-3, lr=np.array([learning_rate/5,learning_rate/3,learning_rate]), dropout = [0.25,0.5]
# F2 :  0.69 (acc 69.35 %) for learning_rate = 1e-2, lr=np.array([learning_rate/5,learning_rate/3,learning_rate]), dropout = [0.25,0.5] # graphs erratic but goes down
# F2 :  0.93 (acc 91.93 %) for learning_rate = 1e-3, lr=np.array([learning_rate/5,learning_rate/3,learning_rate]), dropout = [0.25,0.5] # graphs erratic at the end but goes down
# F2 :  0.94 (acc 93.54 %) for learning_rate = 1e-3, lr=np.array([learning_rate/5,learning_rate/3,learning_rate]), dropout = 0.5


learning_rate = 1e-3
lr=np.array([learning_rate/5,learning_rate/3,learning_rate])
dropout = 0.5
#[0.25,0.5] 


data = ImageDataBunch.from_folder(PATH, ds_tfms=tfms, size=224) # ds_tfms=get_transforms()
data.normalize(imagenet_stats)
learn = ConvLearner(data=data, arch=arch, ps=0.5, pretrained=True, metrics=accuracy)
learn.unfreeze()

training_loop = [
    [123, 64, 10],
    [150, 128, 10],
    [123, 224, 10],
]
for bs, sz, epoch in training_loop:
    data.batch_size = bs
    learn.fit_one_cycle(cyc_len=epoch, max_lr=lr)

# aug_tfms=[RandomRotate(10, tfm_y=TfmType.NO),RandomFlip(),RandomLighting(0.05, 0.05,tfm_y=TfmType.NO), RandomZoom(zoom_max=0.2), RandomStretch(max_stretch=0.2)]
# tfms = tfms_from_model(arch, sz, aug_tfms=aug_tfms, max_zoom=1.1)
# data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=8, num_workers=num_cpus())
# learn = ConvLearner.pretrained(arch, data, precompute=False, ps=0.5)
# learn.unfreeze()
# learn.ps=dropout
# global_results = collections.OrderedDict([])

# training_loop = [
#     [512, 64, 10],
#     [150, 128, 10],
#     [123, 300, 10],
# ]

# for bs, sz, epoch in training_loop:
#     print("Hyperparameters : Batch size=", bs, " Drop out=", dropout, " Learning rate=", learning_rate, " Cycle=", epoch, " Images sizes=", sz )
#     tfms = tfms_from_model(arch, sz, aug_tfms=aug_tfms, max_zoom=0.5)
#     # On recharge les données avec une taille de batch plus importante pour aller plus vite
#     data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=num_cpus())
#     vals_s2s, ep_vals_s2s = learn.fit(lr, n_cycle=epoch, cycle_len=1, get_ep_vals=True)
#     if len(global_results) > 0:
#         for k, v in ep_vals_s2s.items(): global_results[len(global_results)] = v
#     else:
#         global_results = ep_vals_s2s
#     print("After ", str(len(global_results)), " epochs, the accuracy is ", str(vals_s2s[1]*100)[:5], "%" )
#     fichier = arch.__name__ + '_' + str(len(global_results)) + "_" + str(sz) + "_acc" + str(vals_s2s[1]*100)[:5] + '_weights'
#     print("Saving to ", fichier)
#     learn.save(fichier)
# plot_ep_vals(global_results)


In [ ]:
#plot_the_confusion_matrix()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.recorder.plot_losses()

OK, that give us another XXX points improvement !

## Variable cycle length

In [ ]:
!rm -r {PATH}tmp

learning_rate = 1e-3
lr=np.array([learning_rate/10,learning_rate/5,learning_rate])
dropout = 0.5
#F2 :  0.96 for learning_rate = 1e-3, lr=np.array([learning_rate/10,learning_rate/5,learning_rate]), dropout = 0.5
#F2 :  0.92 for learning_rate = 1e-3, lr=np.array([learning_rate/10,learning_rate/5,learning_rate]), dropout = [0.25,0.5]
#F2 :  0.86 for learning_rate = 1e-2, lr=np.array([learning_rate/10,learning_rate/5,learning_rate]), dropout = 0.5
#F2 :  0.88 (acc 90    %) for learning_rate = 1e-3, lr=np.array([learning_rate/5,learning_rate/3,learning_rate]), dropout = 0.5 # graph go slowly down, better result at 29 epoch than 44
#F2 :  0.96 (acc 94.83 %) for learning_rate = 1e-3, lr=np.array([learning_rate/10,learning_rate/5,learning_rate]), dropout = 0.5
#F2 :  0.94 (acc 91.93 %) for learning_rate = 1e-3, lr=np.array([learning_rate/50,learning_rate/10,learning_rate]), dropout = 0.5
#F2 :  0.93 (acc 90.96 %) for learning_rate = 1e-3, lr=np.array([learning_rate/15,learning_rate/6,learning_rate]), dropout = 0.5


# aug_tfms=[RandomRotate(10, tfm_y=TfmType.NO),RandomFlip(),RandomLighting(0.05, 0.05,tfm_y=TfmType.NO), RandomZoom(zoom_max=0.2), RandomStretch(max_stretch=0.2)]
# tfms = tfms_from_model(arch, sz, aug_tfms=aug_tfms, max_zoom=1.1)
# data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=8, num_workers=num_cpus())
# learn = ConvLearner.pretrained(arch, data, precompute=False, ps=0.5)
# learn.unfreeze()
# learn.ps=dropout
# global_results = collections.OrderedDict([])

# training_loop = [
#     [512, 64, 4],
#     [256, 128, 3],
#     [123, 300, 4]
# ]
# for bs, sz, cycle in training_loop:
#     print("Hyperparameters : Batch size=", bs, " Drop out=", dropout, " Learning rate=", learning_rate, " Cycle=", cycle, " Images sizes=", sz )
#     tfms = tfms_from_model(arch, sz, aug_tfms=aug_tfms, max_zoom=0.5)
#     data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=num_cpus())
#     vals_s2s, ep_vals_s2s = learn.fit(lr, cycle, cycle_len=1, cycle_mult=2, get_ep_vals=True)
#     if len(global_results) > 0:
#         for k, v in ep_vals_s2s.items(): global_results[len(global_results)] = v
#     else:
#         global_results = ep_vals_s2s
#     print("After ", str(len(global_results)), " epochs, the accuracy is ", str(vals_s2s[1]*100)[:5], "%" )
#     fichier = arch.__name__ + '_' + str(len(global_results)) + "_" + str(sz) + "_acc" + str(vals_s2s[1]*100)[:5] + '_weights'
#     print("Saving to ", fichier)
#     learn.save(fichier)


In [ ]:
#plt.figure(num=None, figsize=(14, 8), dpi=80, facecolor='w', edgecolor='k')
plot_ep_vals(global_results)


In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.sched.plot_loss()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
learn.save('lesson01-final_model')

Only 2 points improvement but it's great as we are approching Google AutoML performance.

In [ ]:
#loadmodel=False
loadmodel=True
if loadmodel==True:
    data = ImageDataBunch.from_folder(PATH, ds_tfms=get_transforms(), size=224)
    learn = ConvLearner(data=data, arch=arch, ps=0.5, pretrained=True, metrics=accuracy)
    #learn = ConvLearner.pretrained(arch, data, precompute=False, ps=0.5)
    learn.load("lesson01-final_model")
    print("Model loaded")

# Prediction et visualisation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(5,5), dpi=60)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

## Precision and Recall
Précision et rappel en français : https://fr.wikipedia.org/wiki/Pr%C3%A9cision_et_rappel
_La précision est le nombre de documents pertinents retrouvés rapporté au nombre de documents total proposé par le moteur de recherche pour une requête donnée. _ Autrement dit quelle proportion d'identifications positives était effectivement correcte ?

_Le rappel est défini par le nombre de documents pertinents retrouvés au regard du nombre de documents pertinents que possède la base de données._ Autrement dit quelle proportion de résultats positifs réels a été identifiée correctement ?

In [ ]:
data.classes
cm = interp.confusion_matrix()
cm

In [ ]:

# precission = VP / (VP + FP)
precision = cm[0][0]/(cm[0][1]+cm[0][0]);
print("Precision : ", precision)
# recall =  = VP / (VP + FN)
recall = cm[0][0]/(cm[1][0]+cm[0][0]);
print("recall : ", recall)
F1 = 2 * ((precision*recall)/(precision + recall))
print("F1 : ", F1)
F2 = 5 * ((precision*recall)/(4*precision + recall))
print("F2 : ", F2)

In [ ]:
interp.plot_top_losses(16, figsize=(20,20))

C'est bon, les plus petites font plus de 200 pixels.
On note que la majorité on une taille importante, il sera bon de faire des entrainement du modèle avec des tailles supérieures.

_"How many images should we use as a validation set? [01:26:28] Using 20% is fine unless the dataset is small — then 20% is not enough. If you train the same model multiple times and you are getting very different validation set results, then your validation set is too small. If the validation set is smaller than a thousand, it is hard to interpret how well you are doing. If you care about the third decimal place of accuracy and you only have a thousand things in your validation set, a single image changes the accuracy. If you care about the difference between 0.01 and 0.02, you want that to represent 10 or 20 rows. Normally 20% seems to work fine."_
=> Nous avons environs 100 images de validation par classes, ça devrait donc aller.

Faisons un test d'entrainement avec une petite taille de batch pour estimer la valeur du learning rate.